# Evaluation of Segmentation Results

Evaluating segmentation algorithms is most often done using reference data to which you compare your results.

In this notebook, I have written a code for evaluating the segmentation results obtained by the proposed algorithm. 

In this notebook I illustrate the use of the following evaluation criteria:
 
  - Overlap measures:
  - Jaccard and Dice coefficients
  - false negative and false positive errors
  - Surface distance measures:
  - Hausdorff distance (symmetric)
  - mean, median, max and standard deviation between surfaces
  - Volume measures:
  - volume similarity  
    
The relevant criteria are task dependent, so you need to ask yourself whether you are interested in detecting spurious errors or not 
(mean or max surface distance), whether over/under segmentation should be differentiated (volume similarity and Dice or just Dice), 
and what is the ratio between acceptable errors and the size of the segmented object (Dice coefficient may be too sensitive to small 
errors when the segmented object is small and not sensitive enough to large errors when the segmented object is large).


## Importing the python packages and setting the paths

In [1]:
__author__='Somayeh Eb.'

import pandas as pd
import pdb
import numpy as np
import sys
import os
from os.path import isfile,join
from os import listdir

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import scipy.io as sio

from medpy.io import load
import medpy.metric as mdp
import medpy.metric.binary as metrics
import SimpleITK as sitk
import nibabel as nib

import eval_volumes as evol
import eval_segm as es


from enum import Enum
import gui
from ipywidgets import interact, fixed


title_bar=['SubjectID','MeanDSC','MenaIoU','MeanPixelAcc.','ASD(mm)','RMSD(mm)', 'VOE(%)', 'VD(%)','Sensitivity(%)','Specificity(%)','MSD(mm)']

# root directory of subjects' folder
root_dir= '/scratch/gm01/oai-z/all/'

Important Note: change the obj_label for different objects manually based on the object label in the ground truth, 
AND, change the file name accordingly
This code includes the calculation for sensitivity and specificity

In [ ]:
obj_label= 4    # fb =1, fc= 2, tb = 3, tc=4

subjects=listdir(root_dir)
N=len(subjects)
subjects.sort()


# Use enumerations to represent the various evaluation measures
class OverlapMeasures(Enum):
    jaccard, dice, volume_similarity, false_negative, false_positive, sensitivity, specificity = range(7)


class SurfaceDistanceMeasures(Enum):
    hausdorff_distance, mean_surface_distance, std_surface_distance, max_surface_distance, ASSD, MSD, RMSD, VOE, VD = \
        range(9)


# Empty numpy arrays to hold the results
overlap_results = np.zeros((len(subjects), len(OverlapMeasures.__members__.items())))
surface_distance_results = np.zeros((len(subjects), len(SurfaceDistanceMeasures.__members__.items())))

subject_id=[]

overal_mean_dsc=[]
overal_mean_px_acc=[]   #mean pixel accuracy over population
overal_mean_iu=[]
overal_mean_voe=[]
overal_mean_vd=[]

overal_mean_asd=[]
overal_mean_msd=[]
overal_mean_rsd=[]

overal_mean_sn=[]   # overal sensitivity
overal_mean_sp=[]   #overal specificity


## Compute the evaluation criteria

In [ ]:
# Compute the evaluation criteria

# Note that for the overlap measures filter, because we are dealing with a single label we
# use the combined, all labels, evaluation measures without passing a specific label to the methods.
overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()

hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()


for s in range(N):


    sub_dir = root_dir + subjects[s] + '/' + subjects[s] + '_stage1/'
    img_file_dir = sub_dir + subjects[s] + '_data1--data.nii.gz'
    gt_file_dir = sub_dir + subjects[s] + '_data1--label.nii.gz'
    res_file_dir = sub_dir + subjects[s] + '_data1/' + subjects[s] + '_data1--PRED.nii.gz'

    img, img_header=load(img_file_dir)
    i_spacing = img_header.get_voxel_spacing()  #img_header.affine.diagonal()[0:3]  # voxel spacing

    lbl, lbl_header = load(gt_file_dir)  # shape: width,height,depth
    res_vol, res_header = load(res_file_dir)  # shape: width,height,depth

    segmentation_itkimage = sitk.ReadImage(res_file_dir, sitk.sitkUInt8)
    seg = sitk.GetArrayFromImage(segmentation_itkimage)
    seg[seg != obj_label] = 0

    #numpyOrigin = np.array(list(reversed(segmentation_itkimage.GetOrigin())))
    #numpySpacing = np.array(list(reversed(segmentation_itkimage.GetSpacing())))
    numpyOrigin = np.array(list(segmentation_itkimage.GetOrigin()))
    numpySpacing = np.array(list(segmentation_itkimage.GetSpacing()))
    segmentation = sitk.Image(segmentation_itkimage.GetSize(), sitk.sitkUInt8)
    segmentation = sitk.Paste(segmentation, sitk.GetImageFromArray(seg), segmentation_itkimage.GetSize())
    segmentation.SetOrigin(numpyOrigin)
    segmentation.SetSpacing(numpySpacing)

    reference_itkimage = sitk.ReadImage(gt_file_dir, sitk.sitkUInt8)
    ref = sitk.GetArrayFromImage(reference_itkimage)
    ref[ref != obj_label] = 0

    numpyOrigin_ref = np.array(list(reference_itkimage.GetOrigin()))
    numpySpacing_ref = np.array(list(reference_itkimage.GetSpacing()))
    reference = sitk.Image(reference_itkimage.GetSize(), sitk.sitkUInt8)
    reference = sitk.Paste(reference, sitk.GetImageFromArray(ref), reference_itkimage.GetSize())
    reference.SetOrigin(numpyOrigin_ref)
    reference.SetSpacing(numpySpacing_ref)


    # Use the absolute values of the distance map to compute the surface distances (distance map sign, outside or inside
    # relationship, is irrelevant)
    #label = 1
    reference_distance_map = sitk.Abs(
        sitk.SignedMaurerDistanceMap(reference, squaredDistance=False, useImageSpacing=True))
    reference_surface = sitk.LabelContour(reference)

    statistics_image_filter = sitk.StatisticsImageFilter()
    # Get the number of pixels in the reference surface by counting all pixels that are 1.
    statistics_image_filter.Execute(reference_surface)
    num_reference_surface_pixels = int(statistics_image_filter.GetSum())

    # Overlap measures
    overlap_measures_filter.Execute(reference, segmentation)
    overlap_results[s, OverlapMeasures.jaccard.value] = overlap_measures_filter.GetJaccardCoefficient()
    overlap_results[s, OverlapMeasures.dice.value] = overlap_measures_filter.GetDiceCoefficient()
    overlap_results[s, OverlapMeasures.volume_similarity.value] = overlap_measures_filter.GetVolumeSimilarity()
    overlap_results[s, OverlapMeasures.false_negative.value] = overlap_measures_filter.GetFalseNegativeError()
    overlap_results[s, OverlapMeasures.false_positive.value] = overlap_measures_filter.GetFalsePositiveError()
    # Hausdorff distance

    hausdorff_distance_filter.Execute(reference, segmentation)

    surface_distance_results[
        s, SurfaceDistanceMeasures.hausdorff_distance.value] = hausdorff_distance_filter.GetHausdorffDistance()
    # Symmetric surface distance measures
    segmented_distance_map = sitk.Abs(sitk.SignedMaurerDistanceMap(segmentation, squaredDistance=False, useImageSpacing=True))
    segmented_surface = sitk.LabelContour(segmentation)

    # Multiply the binary surface segmentations with the distance maps. The resulting distance
    # maps contain non-zero values only on the surface (they can also contain zero on the surface)
    seg2ref_distance_map = reference_distance_map * sitk.Cast(segmented_surface, sitk.sitkFloat32)
    ref2seg_distance_map = segmented_distance_map * sitk.Cast(reference_surface, sitk.sitkFloat32)

    # Get the number of pixels in the reference surface by counting all pixels that are 1.
    statistics_image_filter.Execute(segmented_surface)
    num_segmented_surface_pixels = int(statistics_image_filter.GetSum())

    # Get all non-zero distances and then add zero distances if required.
    seg2ref_distance_map_arr = sitk.GetArrayViewFromImage(seg2ref_distance_map)
    seg2ref_distances = list(seg2ref_distance_map_arr[seg2ref_distance_map_arr != 0])
    seg2ref_distances = seg2ref_distances + \
                        list(np.zeros(num_segmented_surface_pixels - len(seg2ref_distances)))
    ref2seg_distance_map_arr = sitk.GetArrayViewFromImage(ref2seg_distance_map)
    ref2seg_distances = list(ref2seg_distance_map_arr[ref2seg_distance_map_arr != 0])
    ref2seg_distances = ref2seg_distances + \
                        list(np.zeros(num_reference_surface_pixels - len(ref2seg_distances)))

    all_surface_distances = seg2ref_distances + ref2seg_distances

    # The maximum of the symmetric surface distances is the Hausdorff distance between the surfaces. In
    # general, it is not equal to the Hausdorff distance between all voxel/pixel points of the two
    # segmentations, though in our case it is. More on this below.
    surface_distance_results[s, SurfaceDistanceMeasures.mean_surface_distance.value] = np.mean(all_surface_distances)
    #surface_distance_results[i, SurfaceDistanceMeasures.median_surface_distance.value] = np.median(
    #    all_surface_distances)
    surface_distance_results[s, SurfaceDistanceMeasures.std_surface_distance.value] = np.std(all_surface_distances)
    surface_distance_results[s, SurfaceDistanceMeasures.max_surface_distance.value] = np.max(all_surface_distances)


    #no_slices = img.shape[2]
    no_slices = img.shape[0]   # oai-zib

    print(s, ':   ', subjects[s])
    mean_dsc = np.ndarray(no_slices, dtype=float)
    px_acc = np.ndarray(no_slices, dtype=float)  # define an array to save pixel accuracy
    mean_acc = np.ndarray(no_slices, dtype=float)  # define an array to save mean accuracy
    mean_iu = np.ndarray(no_slices, dtype=float)  # define an array to save mean intersection of union

    mean_voe = np.zeros(no_slices, dtype=float)  # define an array to save volume overlap error
    mean_vd = np.zeros(no_slices, dtype=float)   # define an array to save volume difference

    mean_sn = np.zeros(no_slices, dtype=float)  # define an array to save sensitivity
    mean_sp = np.zeros(no_slices, dtype=float)  ## define an array to save specificity

    blank_index = np.array([])  # an array for keeping blank slice numbers
    #pdb.set_trace()
    for i in range(no_slices):  # for each subjects calculate the mean DSC, pixel accuracy, and IOU
        #segm = res_vol[:, :, i]
        #gt = lbl[:, :, i]
        segm = res_vol[i, :, :]
        gt = lbl[i, :, :]
        segm[segm != obj_label] = 0
        gt[gt != obj_label] = 0

        num_zeros = (gt == 0).sum()
        img_pixs = gt.shape[0] * gt.shape[1]
        if num_zeros == img_pixs:
            blank_index = np.append(blank_index, i)
            continue

        px_acc[i] = es.pixel_accuracy(segm, gt)
        mean_acc[i] = es.mean_accuracy(segm, gt)  #****** I have not added this to the final result*********
        mean_iu[i] = es.mean_IU(segm, gt)
        mean_dsc[i] = es.mean_DSC(segm, gt)
        # mean_voe[i]=es.mean_VOE(segm, gt)
        mean_voe[i] = 100 * (1.0 - np.logical_and(segm, gt).sum() / np.float32(np.logical_or(segm, gt).sum()))
        # mean_voe[i]=100*(1-(mean_dsc[i]/(200-mean_dsc[i])))  # calculating mean volume overalp error for cartilage
        mean_vd[i] = 100 * (segm.sum() - gt.sum()) / float(gt.sum())

        mean_sn[i] = mdp.binary.sensitivity(segm, gt)
        mean_sp[i] = mdp.binary.specificity(segm, gt)

        # overal_mean_voe[s]=np.mean(mean_voe)


    mean_iu = np.delete(mean_iu, blank_index)
    px_acc = np.delete(px_acc, blank_index)  #pixel accuracy
    mean_dsc = np.delete(mean_dsc, blank_index)
    mean_voe = np.delete(mean_voe, blank_index)
    mean_vd = np.delete(mean_vd, blank_index)

    mean_sn = np.delete(mean_sn, blank_index)
    mean_sp = np.delete(mean_sp, blank_index)

    subject_id.append( subjects[s])

    overal_mean_voe.append(np.mean(mean_voe))
    overal_mean_vd.append(np.mean(mean_vd))
    """Calculate surface distance"""
    surface_distance = evol.surfd(res_vol, lbl, i_spacing, 1)
    assd = np.mean((evol.asd(evol.surfd(res_vol, lbl, i_spacing)), evol.asd(evol.surfd(lbl, res_vol, i_spacing))))
    overal_mean_asd.append(assd)


    msd = np.max((evol.msd(evol.surfd(res_vol, lbl, i_spacing)), evol.asd(evol.surfd(lbl, res_vol, i_spacing))))
    overal_mean_msd.append(msd)

    rms = np.sqrt((surface_distance ** 2).mean())
    overal_mean_rsd.append(rms ) # mean vd of subject i

    overal_mean_sn.append(np.mean(mean_sn))
    overal_mean_sp.append(np.mean(mean_sp))

    """
    class OverlapMeasures(Enum):
        jaccard, dice, volume_similarity, false_negative, false_positive, sensitivity, specificity = range(7)


    class SurfaceDistanceMeasures(Enum):
        hausdorff_distance, mean_surface_distance, std_surface_distance, max_surface_distance, ASSD, MSD, RMSD, VOE, VD = \
        range(9)
    """

    overlap_results[s, OverlapMeasures.sensitivity.value] = np.mean(mean_sn)
    overlap_results[s, OverlapMeasures.specificity.value] = np.mean(mean_sp)

    surface_distance_results[s, SurfaceDistanceMeasures.ASSD.value] = assd
    surface_distance_results[s, SurfaceDistanceMeasures.MSD.value] = msd
    surface_distance_results[s, SurfaceDistanceMeasures.RMSD.value] = rms
    surface_distance_results[s, SurfaceDistanceMeasures.VOE.value] = np.mean(mean_voe)
    surface_distance_results[s, SurfaceDistanceMeasures.VD.value] = np.mean(mean_vd)

    #print(overlap_results[0])
    #print(surface_distance_results[0])
    #pdb.set_trace()

#pdb.set_trace()

## Save the results in a csv file


In [ ]:
# Graft our results matrix into pandas data frames
overlap_results_df = pd.DataFrame(data=overlap_results, index = [subjects[s] for s in range(len(subjects))],
                                  columns=[name for name, _ in OverlapMeasures.__members__.items()])
surface_distance_results_df = pd.DataFrame(data=surface_distance_results, index = [subjects[s] for s in range(len(subjects))],
                                  columns=[name for name, _ in SurfaceDistanceMeasures.__members__.items()])

"""with pd.ExcelWriter('V00-FemoralCartilage-OAI.xlsx') as writer:
    overlap_results_df.to_excel(writer, sheet_name = 'overlap_measures')
    surface_distance_results_df.to_excel(writer, sheet_name = 'distance_measures')"""


pd.concat([overlap_results_df,surface_distance_results_df],axis=1).to_csv('TC-OAI-Z-CNN-all.csv')